In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import networkx as nx
from scipy import sparse

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_max_pool, GlobalAttention, GatedGraphConv
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import softmax
from torch_geometric.utils.convert import from_scipy_sparse_matrix
from torch_geometric.data import Data, DataLoader


from pyscf import gto, scf, tools, ao2mo, mp

import pickle
import train
from graph_model import SecondNet, SimpleNet, THCNet
from preprocess import build_qm7, build_thc_graph
from train import train, test
from thc import THCContainer
from utils import khatri_rao

Numpy 1.16 has memory leak bug  https://github.com/numpy/numpy/issues/13808
It is recommended to downgrade to numpy 1.15 or older


In [2]:
basis = 'sto-3g'
# basis = 'cc-pvdz'
mols = build_qm7(basis)
mols = mols[1:10]

In [3]:
print(mols[0].atom)

[['C', (1.8973607, -0.005574692, -0.012094247)], ['C', (4.7544756, -0.013265877, 0.0011149384)], ['H', (1.176071, 1.932056, -0.003099151)], ['H', (1.1604997, -0.9821663, 1.6546631)], ['H', (1.1761088, -0.96077454, -1.6980512)], ['H', (5.4757648, 0.9488315, 1.6830279)], ['H', (5.4913363, 0.9563715, -1.6697053)], ['H', (5.475727, -1.95101, 0.00011338357)]]


In [4]:
kwargs = {'grid_points_per_atom': 3000, 'epsilon_qr': 1e-15, 'epsilon_inv': 1e-15, 'verbose': True,
          'scale': 3.0, 'z_max': 1.0}
mol_data = [THCContainer(mol, kwargs) for mol in mols]

/anaconda3/envs/prime/lib/python3.7/site-packages/pyscf/scf/chkfile.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(chkfile) as fh5:
/anaconda3/envs/prime/lib/python3.7/site-packages/pyscf/lib/misc.py:874: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  h5py.File.__init__(self, filename, *args, **kwargs)


rho L2: 4.445838627142542e-14
T_ao L_infinity: 3.523508510294704e-06
T_mo L_infinity: 2.6301325446453916e-06
T_mo L_2: 1.0065765600761395e-05
rho L2: 2.1157561645206724e-14
T_ao L_infinity: 2.6043965858146834e-05
T_mo L_infinity: 8.908676600416499e-06
T_mo L_2: 3.2648682120230617e-05
rho L2: 2.6496252765719e-14
T_ao L_infinity: 0.00015155260297561313
T_mo L_infinity: 4.056853573186636e-05
T_mo L_2: 0.00017927165012363315
rho L2: 1.2163943932051616e-13
T_ao L_infinity: 0.00025030645400381246
T_mo L_infinity: 0.0002475213191668324
T_mo L_2: 0.0006123149555887334
rho L2: 1.2518959818882955e-13
T_ao L_infinity: 0.0024745384225939304
T_mo L_infinity: 0.002440730170487626
T_mo L_2: 0.00426092280054438
rho L2: 7.196372223623514e-14
T_ao L_infinity: 3.8738598346022e-05
T_mo L_infinity: 3.8471011954754886e-05
T_mo L_2: 0.0001646034906508519
rho L2: 6.07215469900239e-14
T_ao L_infinity: 1.8368001788893196e-05
T_mo L_infinity: 6.4717968690786165e-06
T_mo L_2: 2.979875615605604e-05
rho L2: 1.00562

In [12]:
import matplotlib.pyplot as plt

for (i, con) in enumerate(mol_data):
    
    print(con.M)
    print(con.N)
    
    print(np.max(np.abs(con.Z)))
    print(np.max(np.abs(con.X_mo)))
    
    print(np.sign(con.E[1]))
    print(np.log(con.E[1]))
    print(np.min(con.E[1]))

#     print(con.E[1].shape)
#     data = con.E[1].flatten() + 1e-60
    
#     print(np.min(np.abs(data)))
    
#     _, bins = np.histogram(np.log10(np.abs(data)), bins='auto')
#     plt.hist(data, bins=10**bins);
#     plt.gca().set_xscale("log")
#     plt.show()


16
9
1.0
10.792976129162675
[[1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]]
[[-11.08648952 -11.08647592 -10.13462455 -10.14210335  -9.67767571
  -10.74253814 -10.74252627]
 [-11.0583544  -11.05834148 -10.29852507 -10.03239256  -9.7187415
  -10.92611418 -10.92610248]
 [ -7.00555522  -7.00555093  -6.35710059  -6.42472536  -5.90647968
   -7.04775453  -7.04776884]
 [ -6.85282683  -6.85280397  -6.8120198   -5.80143998  -5.9037984
   -6.86091904  -6.86090648]
 [ -7.22348441  -5.41979362  -7.71765675  -6.64675471  -6.64588938
   -7.17838146  -5.55781719]
 [ -5.4197433   -7.22336163  -7.71813705  -6.64646875  -6.64591625
   -5.55782372  -7.17864007]
 [ -6.94964319  -6.94874275  -4.75922282  -6.2688366   -6.74662478
   -7.49317255  -7.4923034 ]
 [ -6.85218641  -5.4344324   -7.5646037   -6.49366687  -6.40866325
   -7.22866248  -5

/anaconda3/envs/prime/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log
  if sys.path[0] == '':


In [10]:
dataset = []
for con in mol_data:
    
    print("E_J loss", np.linalg.norm(con.E[0] - con.E_THC[0]))
    print("E loss", np.linalg.norm(con.E[2] - con.E_THC[2]))
    print("MP2_J loss", np.linalg.norm(con.MP2[0] - con.MP2_THC[0]))
    print("MP2 loss", np.linalg.norm(con.MP2[2] - con.MP2_THC[2]))
    print(con.E[2].shape)
    print("")
    
    data = build_thc_graph(con)
#     data = Data(X = torch.from_numpy(X), Z = torch.from_numpy(Z),
#                 U = torch.from_numpy(U), coords = torch.from_numpy(coords),
#                 T_ao = torch.from_numpy(T_ao), T_mo = torch.from_numpy(T_mo),
#                mol = mol)
    
    filename = "hello"
    with open(filename, 'wb') as f:
        print(data)
        print(data.con)
        pickle.dump(data, f)
    
    dataset.append(data)

E_J loss 3.7687386890872556e-08
E loss 2.5614485178025037e-08
MP2_J loss 7.936511300932203e-08
MP2 loss 5.1189450220312516e-08
(9, 7)



KeyboardInterrupt: 

In [ ]:
u, v = data.edge_index[:,3000]
u, v = u.item(), v.item()
print(data.x[u])
print(data.x[v])
print(data.edge_attr[3000])

In [ ]:
for data in dataset:
    edge_attr = data.edge_attr
    print(torch.max(edge_attr, dim = 0)[0])
    print(torch.min(edge_attr, dim = 0)[0])
    print(torch.mean(edge_attr, dim = 0))
    
    print()

In [ ]:
import torch.optim as optim

def train(model, loader, lr = 0.003, iterations = 10, verbose = False, lamb = 1.0, device = torch.device("cpu")):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    losses = []
    for i in range(iterations):
        batch_losses = []
        for data in loader:
                        
            E_THC = data.con.E_THC[0] # first term means the J term
            E_THC = torch.from_numpy(E_THC)
            E_hat = model(data)[data.E_mask][:,0].reshape(E_THC.shape)
            E_pred = E_THC + lamb * E_hat
            
            E_true = data.con.E[0] # first term means the J term
            E_true = torch.from_numpy(E_true)
            
            loss = torch.norm(E_true - E_pred) / torch.norm(E_true) #Scale regularization
                    
            optimizer.zero_grad()
            loss.backward()
            
            
            optimizer.step()
                        
            batch_losses.append(loss.item())

        batch_loss = np.mean(np.array(batch_losses))
        losses.append(batch_loss)
        if verbose:
            print("timestep: {}, loss: {:e}".format(i, batch_loss))
    
    model.eval()
    return losses

In [ ]:
vertex_dim = dataset[0].x.shape[1]
edge_dim = dataset[0].edge_attr.shape[1]
hidden_dim = 20
model = THCNet(vertex_dim, edge_dim, hidden_dim).double()

lr = 0.001
verbose = True
lamb = 1e-1

losses = train(model, dataset, iterations = 200, lr = lr, verbose = verbose, lamb = lamb)